### ДЗ 3. Урок 3. Логистическая регрессия. Log Loss#

### -- Автор: Шенк Евгений Станиславович

In [1]:
import numpy as np
import matplotlib.pyplot as plt

### Задание 3.1.

1*. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [2]:
def calc_logloss(y, y_pred, eps=1e-15):
    y_pred = np.clip(y_pred, eps, 1 - eps)
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [3]:
calc_logloss(0, 0)

9.992007221626415e-16

In [4]:
calc_logloss(0, 1)

34.53957599234088

### Задание 3.2.

2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [5]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450,
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 
               1, 3, 3, 1, 2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [6]:
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

In [7]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [8]:
X_st = X.copy()
X_st[2, :] = calc_std_feat(X[2, :])

In [9]:
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
        """if i % (iterations / 10) == 0:
            print(i, W, err)"""
    print(i, W, err)
    return W

In [10]:
alpha = np.logspace(-7, 0, 15)
iterations=10000

In [11]:
for i in alpha:
    print(f'alpha={i}:')
    W = eval_model(X_st, y, iterations=iterations, alpha=i)

alpha=1e-07:
10000 [ 0.49632455 -0.13944699  0.64766909  1.52245353] 1.2181827241996186
alpha=3.162277660168379e-07:
10000 [ 0.49548282 -0.14200309  0.64762671  1.5212083 ] 1.2141158793100248
alpha=1e-06:
10000 [ 0.49282758 -0.1500751   0.64748968  1.51727929] 1.201296577757615
alpha=3.162277660168379e-06:
10000 [ 0.48449745 -0.1754837   0.64702714  1.5049452 ] 1.1611883620416172
alpha=1e-05:
10000 [ 0.45886981 -0.25439705  0.6453131   1.46695998] 1.0393648355410017
alpha=3.1622776601683795e-05:
10000 [ 0.38726319 -0.47258693  0.64075268  1.36320558] 0.743277166616812
alpha=0.0001:
10000 [ 0.25808924 -0.68189547  0.68834749  1.2411954 ] 0.5906743816929662
alpha=0.00031622776601683794:
10000 [ 0.05036748 -0.68544144  0.86152103  1.21416658] 0.5544045544032771
alpha=0.001:
10000 [-0.29756287 -0.72669083  1.06148576  1.39664388] 0.5233563729128639
alpha=0.0031622776601683794:
10000 [-1.00560052 -0.88421229  1.09572777  2.0465512 ] 0.47869105689416525
alpha=0.01:
10000 [-2.77079473 -0.9958

#### Получается, чем крупнее шаг и больше итераций, тем меньше log loss

### Задание 3.3.

3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [12]:
W = np.random.randn(X.shape[0])

In [13]:
def calc_pred_proba(W, X):
    res = 1 / (1 + np.exp(-np.dot(W, X)))
    return res

In [14]:
W = eval_model(X_st, y, iterations=iterations, alpha=1e0)
calc_pred_proba(W, X_st)

10000 [-35.907977    -3.59554341  -9.65060282  29.4178451 ] 0.11663474835747886


array([0.34704715, 0.00982919, 1.        , 0.06771806, 0.93628214,
       0.04710098, 1.        , 0.01781364, 0.58978454, 0.99999976])

### Задание 3.4.

4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [15]:
def calc_pred(W, X, threshold=0.5):
    res = 1 / (1 + np.exp(-np.dot(W, X)))
    y_pred = np.zeros(res.shape[0])
          
    for i in range(res.shape[0]):
        if (res[i] >= threshold): 
            y_pred[i] = 1
        elif (res[i] < threshold):
            y_pred[i] = 0

    return y_pred

In [16]:
W = eval_model(X_st, y, iterations=iterations, alpha=1)
y_pred = calc_pred(W, X_st, 0.5)

10000 [-35.907977    -3.59554341  -9.65060282  29.4178451 ] 0.11663474835747886


### Задание 3.5.

5. Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [17]:
def count_accuracy(y, y_pred):
    return sum(np.where(y == y_pred, np.ones(y.shape[0]), 0)) / y.shape[0]

In [18]:
count_accuracy(y, y_pred)

1.0

In [19]:
def count_err_matrix(y, y_pred):
    TP = sum(np.where((y == y_pred) & (y==1), np.ones(y.shape[0]), 0))
    FP = sum(np.where(y < y_pred, np.ones(y.shape[0]), 0))
    FN = sum(np.where(y > y_pred, np.ones(y.shape[0]), 0))
    TN = sum(np.where((y == y_pred) & (y==0), np.ones(y.shape[0]), 0))
    return np.array([[TP, FP], [FN, TN]])

In [20]:
err_matrix = count_err_matrix(y, y_pred)
err_matrix

array([[5., 0.],
       [0., 5.]])

In [21]:
def count_precision(count_err_matrix):
    return count_err_matrix[0][0] / (count_err_matrix[0][0] + count_err_matrix[0][1])

In [22]:
pr = count_precision(err_matrix)
pr

1.0

In [23]:
def count_recall(count_err_matrix):
    return count_err_matrix[0][0] / (count_err_matrix[0][0] + count_err_matrix[1][0])

In [24]:
rec = count_recall(err_matrix)
rec

1.0

In [25]:
def count_f1_score(precision, recall):
    return 2 * precision * recall / (precision + recall)

In [26]:
count_f1_score(pr, rec)

1.0

### Задание 3.6.

6. Могла ли модель переобучиться? Почему?

Да, могла. При увеличении шага alpha и количества итераций абсолютные значения весов начинают сильно расти, а Accuracy оказалась 100%. Модель построилась максимально заточенная под обучающую выборку.

### Задание 3.7.

7*. Создайте функции eval_model_l1 и eval_model_l2 с применением L1 и L2 регуляризаций соответственно.

In [27]:
def eval_model_l2(X, y, iterations, alpha=1e-4, lambda_=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T) + lambda_ * W)
        """if i % (iterations / 10) == 0:
            print(i, W, err)"""
    print(i, W, err)
    return W

In [28]:
def eval_model_l1(X, y, iterations, alpha=1e-4, lambda_=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T) + (0.5 * lambda_ * np.sign(W)))
        """if i % (iterations / 10) == 0:
            print(i, W, err)"""
    print(i, W, err)
    return W

In [29]:
eval_model_l2(X_st, y, iterations=10000, alpha=1, lambda_=1e-2)

10000 [-2.32119271 -0.76769535  0.52477637  3.46231426] 0.6993767109889983


array([-2.32119271, -0.76769535,  0.52477637,  3.46231426])

In [30]:
eval_model_l1(X_st, y, iterations=10000, alpha=1, lambda_=1e-2)

10000 [-10.38890749  -1.37662516  -2.08786423   8.77497892] 0.261875535513877


array([-10.38890749,  -1.37662516,  -2.08786423,   8.77497892])

In [31]:
eval_model(X_st, y, iterations=10000, alpha=1)

10000 [-35.907977    -3.59554341  -9.65060282  29.4178451 ] 0.11663474835747886


array([-35.907977  ,  -3.59554341,  -9.65060282,  29.4178451 ])

In [32]:
eval_model_l2(X_st, y, iterations=10000, alpha=1e-2, lambda_=1e-2)

10000 [-1.78523476 -0.79441842  0.6154667   2.35642915] 0.44900175454779107


array([-1.78523476, -0.79441842,  0.6154667 ,  2.35642915])

In [33]:
eval_model_l1(X_st, y, iterations=10000, alpha=1e-2, lambda_=1e-2)

10000 [-2.4911916  -0.91132241  0.52415355  2.96814499] 0.41694361826840376


array([-2.4911916 , -0.91132241,  0.52415355,  2.96814499])

In [34]:
eval_model(X_st, y, iterations=10000, alpha=1e-2)

10000 [-2.77079473 -0.99580928  0.56650766  3.2676589 ] 0.4058305387773311


array([-2.77079473, -0.99580928,  0.56650766,  3.2676589 ])